# CAPSIF Prediction on a directory #

### Input your directories below - same as seen in `predict_directory.py` ###
### Results outputted to __`[out_dir]/*.pdb`__ with predictions and the file __`[out_dir]/capsif_predictions.txt`__ ###

## Current settings for B Factor visualization: ##
#### `BFactor =  0.0` : Nonbinder ####
#### `BFactor = 40.0` : CAPSIF:G Predicted Binder ####
#### `BFactor = 59.9` : CAPSIF:V Predicted Binder ####
#### `BFactor = 99.9` : CAPSIF:V and CAPSIF:G Predicted Binder ####

## Changeable Parameters ##

`input_dir` - input directory of pdbs for prediction

`out_dir` - output directory of predictions

`graph_model_dir` - graph model of choice

`voxel_model_dir` - voxel model of choice

`make_pdb` - whether or not to output the pdb's for visualization

## Change the input values then RUN ALL to get your results!!! ##

In [1]:
input_dir = "./sample_dir/" #Enter input directory here
out_dir = "./sample_dir/output/" #If not using default graph model, input your graph model here
graph_model_dir = "./capsif_g/models_DL/cb_model.pth.tar" #If not using default graph model, input your CAPSIF:G model here
voxel_model_dir = "./capsif_v/models_DL/my_checkpoint_best_36_2A_CACB_vector_coord_I_clean_data.pth.tar" #If not using default CAPSIF:V model, input your graph model here
make_pdb = True; #Output the files with the B factor as 99.99 for when bound

### Initialize Directories and libraries ###

In [2]:
#Safety check with rerunning things to switch back to the native absolute path :)
import os
native_dir = os.path.abspath('.') + "/"
input_dir = os.path.abspath(input_dir) + "/"
out_dir = os.path.abspath(out_dir)
graph_model_dir = os.path.abspath(graph_model_dir)
voxel_model_dir = os.path.abspath(voxel_model_dir)

In [3]:
import sys

os.chdir(native_dir)

os.chdir("./capsif_v/")
#CAPSIF:V requirements
sys.path.append("./capsif_v/")
from capsif_v.utils import xyz_to_rotate_to_voxelize_to_translate
from capsif_v.prediction_utils import load_model as load_voxel_model, command_run
from data_preparation.pdb_2_interaction_file_converter import pdb_to_interaction_file as capsif_v_pdb_preprocess
os.chdir('../')
sys.path.append('../')

os.chdir("./capsif_g/")
sys.path.append("./capsif_g/")
from capsif_g.dataset import load_predictor_model as load_graph_model
from notebook_library import download_pdb, predict_for_voxel as predict_voxel, visualize, preprocess_graph, predict_for_graph as predict_graph, output_structure_bfactor_biopython_BOTH as output_structure_bfactor
os.chdir("../")
sys.path.append("../")

import numpy as np
import time
import torch


#Directory information!
input_dir = os.path.abspath(input_dir) + "/"
out_dir = os.path.abspath(out_dir) + "/"


### Initialize Models ###

In [4]:
#Initialize Models

#Run only on CAPSIF:V files
# Initialize pdb into npz file for reading
#if you fail to get outputs, feel free to mess around with the values :)
pdb_npz_file_reader = xyz_to_rotate_to_voxelize_to_translate()
pdb_npz_file_reader.max_rotation_plus_minus = 0
pdb_npz_file_reader.max_pixel_translate_per_axis = 0
pdb_npz_file_reader.use_res_index = 1
pdb_npz_file_reader.layers_use = 29
pdb_npz_file_reader.layer_29_type = 2
pdb_npz_file_reader.cube_start_points = 1
pdb_npz_file_reader.crop_extra_edge = 0
pdb_npz_file_reader.cube_start_points = 1

#load model
os.chdir('capsif_v')
start_time = time.time()
model = load_voxel_model(2,2,dir=voxel_model_dir)
model_time = time.time()
print("CAPSIF:V Model load time: ","%5.1f " % (model_time -start_time), "seconds.\n")
os.chdir(native_dir)

#Initialize Graph
start_time = time.time()
graph_model = load_graph_model(graph_model_dir)
model_time = time.time()
print("CAPSIF:G Model load time: ","%5.3f " % (model_time -start_time), "seconds.")

Using /Users/scanner1/Downloads/capsif_repo/CAPSIF/capsif_v/models_DL/my_checkpoint_best_36_2A_CACB_vector_coord_I_clean_data.pth.tar
=> Loading checkpoint
CAPSIF:V Model load time:    1.2  seconds.

CAPSIF:G Model load time:  0.035  seconds.


## Run Prediction ##

In [5]:
ls = os.listdir(input_dir);

out_file = open(out_dir + 'capsif_predictions.txt','w+')
out_file.write("PDB_Name,CAPSIF:V_predictions,CAPSIF:G_predictions\n")

for pdb_file in ls:
    if '.pdb' in pdb_file:
        
        os.chdir('capsif_v')
        print('Currently treating: ' + pdb_file)
                
        f = capsif_v_pdb_preprocess( input_dir + pdb_file, input_dir,
                                            0, verbose=0,
                                            use_glycan=1)
        f.carb_aa_distance_calc_and_save = 1
        f.save_data =0
        s,x = f.run_me()

        if len(x) <= 1:
            if len(x) == 0:
                print("Can not read pdb file! Not a protein data.")

            if x[0] == -1:
                print("PyRosetta could not read glycan!")
                print("Check PDB file (ring glycan, clashes etc.)!")
                print("or use restart the code and run 'load_glycan_off' flag. Dice will be 0")

        #CAPSIF:V Predictions
        proteins,masks = pdb_npz_file_reader.apply(x,s)
        d,voxel_res,py_com,pred_vox=predict_voxel(torch.from_numpy(proteins), torch.from_numpy(masks), model,  2 ,save_npz=0)
        os.chdir(native_dir)
        
        #CAPSIF:G Predictions
        os.chdir('capsif_g/')
        graph_dice, graph_results, graph_res = predict_graph(input_dir + pdb_file,model=graph_model)
        os.chdir(native_dir)
        

        if make_pdb :
            output_structure_bfactor(in_file=input_dir + pdb_file,res_v=voxel_res,res_g=graph_res,out_file=out_dir + pdb_file[:pdb_file.index('.')] + "_predictions.pdb")
        
        out_file.write(pdb_file + " : " + voxel_res + '\n')

out_file.close()

print("CAPSIF Predictions Finished!")
print("Outputted to: " + out_dir + ".txt")

Currently treating: 4EXK_1_AF.pdb
PyRosetta-4 2021 [Rosetta PyRosetta4.Release.python38.mac 2021.36+release.57ac713a6e1d8ce6f60269b3988b1adac1d96fc6 2021-09-10T13:50:04] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
PyRosetta-4 2021 [Rosetta PyRosetta4.Release.python38.mac 2021.36+release.57ac713a6e1d8ce6f60269b3988b1adac1d96fc6 2021-09-10T13:50:04] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
Currently treating: 3W26.pdb
PyRosetta-4 2021 [Rosetta PyRosetta4.Release.python38.mac 2021.36+release.57ac713a6e1d8ce6f60269b3988b1adac1d96fc6 2021-09-10T13:50:04] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
PyRosetta-4 2021 [Rosetta PyRosetta4.Release.python38.mac 2021.36+release.57ac713a6e1d8ce6f60269b

/Users/scanner1/opt/anaconda3/lib/python3.8/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 3014.
  warnings.warn(
